In [ ]:
import cv2
import time
import os
import math
import sys
import numpy as np

from skimage.morphology import skeletonize
from skimage.measure import find_contours
from scipy.ndimage import convolve

sys.path.append("../modules/")

import input_output
import processor
import image_processing

det = processor.Processors ()
det.add_processor ("fingers_counting")

det.add_filter (processor.colorspace_to_colorspace ("RGB", "HSV"),
                "fingers_counting", "colorspace change")

low_th  = (50, 0, 145)
high_th = (255, 255, 255)
det.add_filter (processor.inrange (low_th, high_th), "fingers_counting", "inrange rgb")

cv2.namedWindow ('trackbars')

start_val = list (low_th) + list (high_th)

cv2.createTrackbar ("lr", "trackbars", start_val [0], 255, 
    lambda new_coeff : det.processors ["fingers_counting"]
    ["inrange rgb"].set_th (new_coeff, "low", 0))

cv2.createTrackbar ("hr", "trackbars", start_val [3], 255, 
    lambda new_coeff : det.processors ["fingers_counting"]
    ["inrange rgb"].set_th (new_coeff, "high", 0))

cv2.createTrackbar ("lg", "trackbars", start_val [1], 255, 
    lambda new_coeff : det.processors ["fingers_counting"]
    ["inrange rgb"].set_th (new_coeff, "low", 1))

cv2.createTrackbar ("hg", "trackbars", start_val [4], 255, 
    lambda new_coeff : det.processors ["fingers_counting"]
    ["inrange rgb"].set_th (new_coeff, "high", 1))

cv2.createTrackbar ("lb", "trackbars", start_val [2], 255, 
    lambda new_coeff : det.processors ["fingers_counting"]
    ["inrange rgb"].set_th (new_coeff, "low", 2))

cv2.createTrackbar ("hb", "trackbars", start_val [5], 255, 
    lambda new_coeff : det.processors ["fingers_counting"]
    ["inrange rgb"].set_th (new_coeff, "high", 2))

det.add_filter (processor.morphology ("open", 5), "fingers_counting", "morphological opening")
det.add_filter (processor.leave_max_area_cc (), "fingers_counting", "leave max area cc")

det.add_filter (processor.custom_operation (lambda img: image_processing.fill_holes (img),
                "to_simply_connected"), "fingers_counting", "to simply connected")

det.add_filter (processor.custom_operation (lambda img: cv2.medianBlur (img, 23),
                "median_blur"), "fingers_counting", "median blur")

additional_stages = []

def extract_and_filter_endpoints (img):
    global additional_stages
    
    def detect_endpts(skeleton):

        end_kernel = [[1,  1, 1],
                      [1, 10, 1],
                      [1,  1, 1]]

        detected = convolve(skeleton, end_kernel)
        where_detected = np.where(detected == 11)
        end_points = [[where_detected[0][i], where_detected[1][i]] for i in
                      range(where_detected[0].shape[0])]

        return end_points

    def get_contour(binary):
        # returns image of contour of shape from <binary> as the 1st output
        # and a list of contour points as the 2nd output

        contour_points = find_contours(binary, 0.5)
        if contour_points:
            contour_points = contour_points[0].astype(int)
        else:
            return -1

        contour = np.zeros_like(binary)
        
        for point in contour_points:
            contour[point[0]][point[1]] = 1
        
        return contour

    def remove_endpts(contour, end_points, kern_sz=65, min_score=5):
        assert kern_sz % 2 == 1, "kern_sz should be odd number!"
        
        if (isinstance(contour, int)):
            return []

        height, width = contour.shape

        # create star-like kernel
        kernel = np.zeros((kern_sz, kern_sz), dtype='int')
        kernel[kern_sz // 2, :] = 1
        kernel[:, kern_sz // 2] = 1
        for i in range(kern_sz):
            kernel[i, i] = 1
            kernel[i, kern_sz - i - 1] = 1

        scores = []
        for pt in end_points:
            score_penalty = 0
            
            if (pt[0] >= height - 5 or
                pt[0] <= 5 or
                pt[1] >= width - 5 or
                pt[1] <= 5):
                score_penalty = 10
            
            x1 = pt[1] - kern_sz // 2
            y1 = pt[0] - kern_sz // 2
            x2 = x1 + kern_sz
            y2 = y1 + kern_sz
            x1_shift, x2_shift, y1_shift, y2_shift = 0, 0, 0, 0
            if x1 < 0:
                x1_shift = -x1
                x1 = 0
            if x2 > width:
                x2_shift = x2 - width
                x2 = width
            if y1 < 0:
                y1_shift = -y1
                y1 = 0
            if y2 > height:
                y2_shift = y2 - height
                y2 = height

            finger = contour[y1:y2, x1:x2]
            cropped_kernel = kernel[y1_shift:kern_sz - y2_shift, x1_shift:kern_sz - x2_shift]
            scores.append((finger * cropped_kernel).sum() - score_penalty)

        end_points = [pt for pt, score in zip(end_points, scores) if score >= min_score]

        return end_points
    
    skeleton = skeletonize (img / 255).astype(np.uint8)
    endpoints = detect_endpts (skeleton)
    
    cont = get_contour (img / 255)
    
    endpoints = remove_endpts (cont, endpoints)
    
    notgray = cv2.cvtColor (img, cv2.COLOR_GRAY2RGB)
    
    for endpoint in endpoints:
        cv2.circle (notgray, (endpoint [1], endpoint [0]), 10, (100, 200, 3), -1)

    notgray = cv2.putText (notgray, str(len (endpoints)), (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 255, 0), 7, cv2.LINE_AA)
    
    if (isinstance(cont, int)):
        additional_stages = [skeleton * 255, notgray, notgray]
    
    else:
        additional_stages = [skeleton * 255, cont.astype(np.uint8) * 255, notgray]
    
    return notgray

det.add_filter (processor.custom_operation (lambda img: extract_and_filter_endpoints(img),
                "endpoints_extraction"), "fingers_counting", "endpoints extraction")

source = input_output.Source ("../data/hand_gestures2.mkv")
output = input_output.Writer ("output.mp4", 1348, 759, 7)

while (True):
    _, frame = source.get_frame ()

    mask, success = det.process (frame, "fingers_counting")

    stages = det.get_stages_picts ("fingers_counting") [:-1]
    
    if (len (additional_stages) == 3):
        stages += [additional_stages [0]]

        if (not isinstance(additional_stages [1], int)):
            stages += [additional_stages [1]]

        stages += [additional_stages [2]]
        
    output_frame = input_output.form_grid (stages, 1350)
    cv2.imshow ("frame", output_frame)
    output.write (output_frame)
    
    time.sleep (0.02)

    keyb = cv2.waitKey (1) & 0xFF
    
    if (keyb == ord('q')):
        break

source.release()
output.release()

cv2.waitKey (0)
cv2.destroyAllWindows()

video


In [ ]:
Нужно вводить ветвление: каждый фильтр должен принимать, что ему нужно на вход. Это
требуется, чтобы 

Нужно вводить ветвление. Визуализация возможна через стрелки от предка к потомку.

Структурно зрение очень напоминает весть проект целиком, то есть сборку
конфигурации из модулей. Получается, что система сборки должна быть одной и той же,
а модули разными.

При этом система сборки может быть включена в один из модулей (зрение).
Ассемблер (или скорее контейнер, который всех крутит и обновляет) получает на вход конфиг.
Все модули должны уметь из коробки визуализировать, что они делают!!

Модуль:
инит (лист с входами)
работатб ()

Конфиг:

модуль1:
    config_path : smth.json
    direct_config : конфиг, как например у зрения
    inputs: []

модуль2:
    config_path : smth.json
    direct_config : конфиг, как например у зрения
    inputs: []

Класс Container должен работать так же, как и все остальные. Хочется визуализации графа обработки данных, то есть кто кому дату отдает

In [ ]:
Можно сделать у Source три режима чтения данных - реальный сенсор, симулятор, диск